In [ ]:
# generate psth_stim, psth_stim_meta, psth_scenefile, psth_scenefile_meta

In [1]:
from natsort import os_sorted
import pickle
import pathlib as Path
import os
import sys
from sys import platform
import pandas as pd
import numpy as np
from utils_ephys import * 
from utils_meta import * 



In [2]:
base_data_path = Path.Path('/Volumes/issa-locker/Data')
base_save_out_path = Path.Path('/Volumes/issa-locker/users/Younah/ephys/')
monkey = 'West'
date = '20231206'

In [3]:
data_path, save_out_path, plot_save_out_path = init_dirs(base_data_path, monkey, date, base_save_out_path)

1  recordings found
/Volumes/issa-locker/Data/West/West_20231206_R_H01_P06/West_20231206_R_H01_P06_AP03.2_DV06.2_ML12.7_Ang50.0_Dep05200-05200_g0/West_20231206_R_H01_P06_AP03.2_DV06.2_ML12.7_Ang50.0_Dep05200-05200_g0_imec0

Data path found: True
All works will be saved to
/Volumes/issa-locker/users/Younah/ephys/West/West_20231206_R_H01_P06
All plots will be saved to 
/Volumes/issa-locker/users/Younah/ephys/West_plots/West_20231206_R_H01_P06


In [4]:
print(save_out_path)

/Volumes/issa-locker/users/Younah/ephys/West/West_20231206_R_H01_P06


In [ ]:
MUA_dir = data_path / Path.Path('MUA_4SD')
data_dict_path = os_sorted(save_out_path.glob('data_dict_' + monkey + '*'))[0]
stim_info_path= os_sorted(save_out_path.glob('stim_info_sess'))[0]
n_chan = 0

In [ ]:
# get baseline activity

ch_psth_bl, ch_psth_bl_meta,ch_psth_bl_stim = get_data_bl(n_chan, MUA_dir, data_dict_path,stim_info_path)

pickle.dump(ch_psth_bl, open(save_out_path /'ch{:0>3d}_psth_bl'.format(n_chan) ,'wb'), protocol = 2)
pickle.dump(ch_psth_bl_stim, open(save_out_path /'ch{:0>3d}_psth_bl_stim'.format(n_chan) ,'wb'), protocol = 2)


In [ ]:
# get spike times, peak amplitude, waveform, and psth per stimulus 

ch_pk_stim, ch_wf_stim, ch_st_stim, ch_sl_stim, ch_ind_stim,ch_psth_stim,ch_psth_stim_meta = \
    get_data_bystim(n_chan, MUA_dir, stim_info_path, binwidth_psth= 0.01)

pickle.dump(ch_pk_stim, open(save_out_path / 'ch{:0>3d}_pk_stim'.format(n_chan),'wb'), protocol = 2)
pickle.dump(ch_wf_stim, open(save_out_path / 'ch{:0>3d}_wf_stim'.format(n_chan),'wb'), protocol = 2)
pickle.dump(ch_st_stim, open(save_out_path / 'ch{:0>3d}_st_stim'.format(n_chan),'wb'), protocol = 2)
pickle.dump(ch_sl_stim, open(save_out_path / 'ch{:0>3d}_sl_stim'.format(n_chan),'wb'), protocol = 2)
pickle.dump(ch_ind_stim, open(save_out_path / 'ch{:0>3d}_ind_stim'.format(n_chan),'wb'), protocol = 2)
pickle.dump(ch_psth_stim, open(save_out_path /'ch{:0>3d}_psth_stim'.format(n_chan),'wb'), protocol = 2)
pickle.dump(ch_psth_stim_meta, open(save_out_path /'ch{:0>3d}_psth_stim_meta'.format(n_chan),'wb'), protocol = 2)

# get mean psth per stimulus
# get the longest length 
psth_len= 0
for stim in ch_psth_stim:
    psth_len = np.nanmax((psth_len,ch_psth_stim[stim].shape[1]))

# get the longest length
mean_psth = np.nan * np.ones((len(ch_psth_stim), psth_len))
for n,stim in enumerate(ch_psth_stim):
    mean_psth[n,0:ch_psth_stim[stim].shape[1]] = np.nanmean(ch_psth_stim[stim],axis = 0)

np.save(save_out_path / 'ch{:0>3d}_mean_psth.npy'.format(n_chan),mean_psth)


In [ ]:
# get psth per scenefile
# stimulus in a session
ch_psth_stim = pickle.load(open(save_out_path / 'ch{:0>3d}_psth_stim'.format(n_chan),'rb'))
ch_psth_stim_meta = pickle.load(open(save_out_path / 'ch{:0>3d}_psth_stim_meta'.format(n_chan),'rb'))

stim_list_sess = list(ch_psth_stim_meta.keys())

all_scenefile = []
for stim in stim_list_sess:
    all_scenefile.extend(ch_psth_stim_meta[stim]['scenefile'])

unique_scenefile = np.unique(all_scenefile)

print(unique_scenefile)

psth_byscenefile = dict.fromkeys(unique_scenefile) 
psth_byscenefile_meta = dict.fromkeys(unique_scenefile)

for s in unique_scenefile:
    psth_new = []
    psth_byscenefile_meta[s] = dict()
    psth_byscenefile_meta[s]['stim_ids'] = []
    psth_byscenefile_meta[s]['ind'] = []
    psth_byscenefile_meta[s]['n_trials_stim'] = []
    n_trials = 0
    for stim_ind,stim in enumerate(stim_list_sess):
        if s in ch_psth_stim_meta[stim]['scenefile']:
            psth_new.append(ch_psth_stim[stim])
            psth_byscenefile_meta[s]['ind'].append(stim_ind)
            psth_byscenefile_meta[s]['stim_ids'].append(stim)
            t_before = ch_psth_stim_meta[stim]['t_before']
            t_after = ch_psth_stim_meta[stim]['t_after']
            psth_bins = ch_psth_stim_meta[stim]['psth_bins']
            binwidth = ch_psth_stim_meta[stim]['binwidth']
            n_trials += ch_psth_stim_meta[stim]['n_trials']
            psth_byscenefile_meta[s]['n_trials_stim'].append(ch_psth_stim_meta[stim]['n_trials'])
            stim_dur = ch_psth_stim_meta[stim]['stim_dur']
    psth_new = np.vstack(psth_new)
    psth_byscenefile[s]= psth_new
    psth_byscenefile_meta[s]['stim_dur'] = stim_dur
    psth_byscenefile_meta[s]['t_before'] = t_before
    psth_byscenefile_meta[s]['t_after'] = t_after
    psth_byscenefile_meta[s]['psth_bins'] = psth_bins
    psth_byscenefile_meta[s]['binwidth'] = binwidth
    psth_byscenefile_meta[s]['n_trials'] = n_trials

pickle.dump(psth_byscenefile, open(save_out_path /'ch{:0>3d}_psth_scenefile'.format(n_chan),'wb'), protocol = 2)
pickle.dump(psth_byscenefile_meta, open(save_out_path /'ch{:0>3d}_psth_scenefile_meta'.format(n_chan),'wb'), protocol = 2)